# 1. Inleiding

In *Data Understanding* hebben we alle ingrediënten verzameld die nodig zijn om de businessvraag op basis van de data te beantwoorden. Echter, dat beantwoorden doen we niet in een notebook. Een notebook is handig om onderzoek te doen en tijdens het onderzoek je resultaten vast te leggen en te documenteren. Om een herhaalbare en betrouwbare analyse te kunnen doen, is het beter om met Python code in modules (of scripts) te werken. 

# 2. Mappenstructuur project

De scripts vind je in de map `\src\` van deze repository.

De structuur die meestal voor een dergelijk project wordt gebruikt:

```
projectmap:
│
│   .gitignore          Specificeert welke mappen/bestanden niet in Git worden opgenomen
│   README.md           Hoofpagina voor GitHub-repository
│   requirements.txt    Specificeert de packages (met versienummers) voor de virtuele environment
│       
├───data
│       De geëxporteerde CSV-bestanden
│       
├───notebooks
│       Jupyter Notebooks van het project
│       
├───pentaho
│       Pentaho transformaties en job (*.ktr, *.kjb)
│       
├───powerbi
│       Power BI-dashboards (*.pbix)
│       
├───src
│       avgspeed.py         Export lap-snelheden
│       driverdata.py       Export coureurdata
│       main.py             Hoofdbestand 
│       racecalendar.py     Export racekalender
│       settings.py         Project-instellingen
│       utils.py            Hulpfuncties
│       __init__.py         Maakt van map een Python-package
│       
└───tests
        Plaats voor test-scripts

```

# 3. Gewenste architectuur

In de figuur is een schets van de architectuur weergegeven.

![](figures/architectuur.svg)

We gaan uit van een Windows pc met daarop Python beschikbaar. Met drie Python-scripts wordt Fast F1-data geladen met behulp van de [Fast F1 API](https://docs.fastf1.dev/). De data wordt geëxporteerd naar drie CSV-bestanden. Deze bestanden worden met [Pentaho Data Integration](https://www.hitachivantara.com/en-us/products/pentaho-platform/data-integration-analytics/pentaho-community-edition.html) in een PostgreSQL-database `fastf1_staging` geladen. Vervolgens wordt de geladen data met Pentaho getransformeerd en geladen naar de database `fastf1_dwh`.

Het geheel (ook de Python-scripts) wordt met een Pentaho Job aangestuurd. Deze job zou kunnen worden 'gescheduled' zodat het geheel wordt geautomatiseerd.

# 4. Beschrijving Python-scripts

## Disclaimer

De scripts bevatten diverse Python-constructies die niet in het eerste jaar bij *Programming* zijn behandeld. Ze worden hier ook verder niet echt toegelicht omdat het hier niet per se om het programmeren in Python gaat. 

## `main.py`

`main.py` is het script dat vanuit Pentaho wordt uitgevoerd. Het roept de andere scripts aan, zodat de koppeling met Pentaho zo eenvoudig mogelijk is. Het script is heel eenvoudig omdat de complexiteit in de andere scripts zit.

## `settings.py`

`settings.py` regelt alle instellingen voor het project. De andere scripts maken er gebruik. Het beheer van een project is veel eenvoudiger wanneer je één centrale plek gebruikt voor alle instellingen. Als we bijvoorbeeld het aantal jaren waarover we de analyse doen, willen aanpassen, hoeven we dat alleen maar in `settings.py` aan te passen. 

## `utils.py`

`utils.py` bevat een paar algemene handige functies die de andere scripts ook gebruiken. Door ze in één module te zetten, hoef je geen code te kopiëren, maar kun je ze gebruiken waar je ze nodig hebt.

## `avgspeed.py`

`avgspeed.py` is het meest gecompliceerde script. In het script zijn acht functies gedefinieerd. Iedere functie heeft een eigen specifieke taak. Door de complexiteit zo op te splitsen in kleine overzichtelijke taken, wordt het geheel makkelijker te begrijpen en te debuggen. De belangrijkste functie is `main()`. Deze functie roept de andere functies aan.

Onderaan het bestand zie je de volgende code:

```python
if __name__ == "__main__":
    main(NUM_OF_YEARS)
```
Dit zorgt ervoor dat de functie `main()` op de juiste manier wordt aangeroepen wanneer je het script op zichzelf zou uitvoeren (bv. via de command line met `python -m main.py`). Zonder deze regels code zou het script alleen een aantal functies definiëren, maar geen enkele functie uitvoeren.

De meest ingewikkelde functies zijn `get_race_events()` en `prep_for_export()`. In beide functies wordt gebruikgemaakt van een zogenaamde *list comprehension*, dat is een soort `for`-lus in één commando. In `prep_for_export()` wordt ook nog gebruikgemaakt van *tuple unpacking* (`*lp`) en van de `zip`-functie van Python. De `_` in `(_, lp)` is een korte variabelenaam die Python-programmeurs vaak gebruiken om een dummywaarde aan te duiden (een variabele die niet wordt gebruikt).

## `driverdata.py`

`driverdata.py` haalt de coureursinformatie op van de `Ergast.com` website. Dit wordt gedaan met behulp van het `requests`-package in de functie `get_drivers_in_year()`. Omdat web requests foutgevoelig kunnen zijn (bv. door veel internetverkeer) is er een beschermingsmechanisme ingebouwd. In de `for`-lus wordt een aantal keer achter elkaar geprobeerd een request te doen, totdat de respons-code goed is. De wachttijd tussen twee pogingen wordt exponentieel groter (*exponential back-off*) om te voorkomen dat de server onnodig wordt belast.

De Ergast-API geeft een JSON-bestand terug. Daarin zit (een beetje diep verscholen) een tabel met de coureursinformatie (`DriverTable`). Dat is de tabel die wordt geëxporteerd.

Ook hier is de `main`-functie de belangrijkste functie en is er onderaan het bestand een aantal regels die ervoor zorgen dat `main` wordt aangeroepen als het bestand zelfstandig wordt uitgevoerd.

## `racecalendar.py`

`racecalendar.py` is het eenvoudigste script, hoewel in de `main`-functie binnen de aanroep van `pd.concat()` een *generator expression* wordt gebruikt. Dit is vergelijkbaar met de *list comprehension*, een soort `for`-lus in één regel.

# 5. ETL met Pentaho

## 5.1 Introductie Pentaho

We zouden in principe het hele ETL-proces in Python kunnen programmeren, dus ook het importeren van de CSV-bestanden in de staging-database, het transformeren naar een sterschema en het importeren in het datawarehouse. Toch is dat niet de route die we kiezen. Het is altijd goed om de juiste tool voor de juiste taak in te zetten. Het importeren van data met een API is typisch iets wat heel goed gaat met Python, maar het laden van CSV-bestanden, data wegschrijven in een database en data uit een database transformeren is typisch iets waar ETL-tools voor gemaakt zijn. Daarom kiezen we voor de rest van de flow voor [Pentaho Data Integration](https://www.hitachivantara.com/en-us/products/pentaho-platform/data-integration-analytics/pentaho-community-edition.html) (PDI), een ETL-tool. 

PDI (community edition) is een professionele ETL-tool die je legaal gratis kunt downloaden. Er wordt geen apart installatieprogramma bij geleverd, dus je moet het zelf installeren (= ZIP-bestand uitpakken en zorgen dat Java beschikbaar is). Om PDI te kunnen draaien moet een 64-bit-versie van Java SE (versienummer tussen 11 en 18) op je computer geïnstalleerd zijn. De installatiestappen vind je [hier](https://www.hitachivantara.com/en-us/pdf/implementation-guide/three-steps-to-install-pentaho-data-integration-ce.pdf).

In Pentaho kun je twee dingen aanmaken:
- **Transformaties (`*.ktr`)** Een transformatie is een specificatie van een datastroom. In andere ETL-tools wordt het ook wel een *mapping* genoemd. Je plaatst pictogrammen op een canvas en verbindt ze met pijlen. De pictogrammen staan voor operaties (lezen, transformeren of schrijven) die je verbindt met pijlen. De pijlen kun je zien als leidingen waar data door stroomt. Het configureren van de operaties kan soms wat technisch zijn (ETL-tools zijn bijvoorbeeld erg kieskeurig als het om datatypen gaat) maar verder is het vrij eenvoudig om een transformatie te maken. De transformatie kun je vervolgens met de playknop uitvoeren. Je kunt het uitvoeren ook geautomatiseerd laten plaatsvinden door een *job* te gebruiken. 
- **Jobs (`*.kjb`).** Een job is de specificatie van een proces of stappenplan. Hiermee kun je transformaties, scripts of andere operaties in een gewenste volgorde uitvoeren. Ook kun je controles toevoegen zodat je op foutsituaties kunt inspelen. Een job zelf kan ook weer een andere job aanroepen of geautomatiseerd laten uitvoeren. In een productie-omgeving worden ETL-jobs vaak één keer per nacht uitgevoerd om data te laden en om de data in het datawarehouse bij te werken.

We gaan hieronder een aantal dingen doen:

1. Databases aanmaken
2. De CSV-bestanden inlezen en naar `fastf1_staging` schrijven
3. Een sterschema ontwerpen
4. Transformaties maken voor het vullen van `fastf1_dwh`
5. Jobs aanmaken om het geheel te automatiseren

## 5.2 Stap 1: Data importeren in de `fastf1_staging`-database

Voordat je de Pentaho-transformaties moet je de beide databases aanmaken. Dat gaat als volgt:

### `fastf1_staging`
Open een PowerShell-venster in de map `pentaho\sql` en voer het volgende commando uit (let op hoofdletters en kleine letters!):
```powershell
psql -U <db_username> -W -h <host> -p <port> -f fastf1_staging.sql
```
Vervang hierin: 
- `<db_username>` door je PostgreSQL-gebruikersnaam (meestal `postgres`)
- `<host>` door de naam van de PostgreSQL-server (meestal `localhost`)
- `<port>` door de PostgreSQL-poort (meestal `5432`)

*Let op!* Je moet twee keer je (PostgreSQL-)wachtwoord ingeven.

### `fastf1_dwh`

Hetzelfde commando maar nu met het andere `.sql`-bestand:

```powershell
psql -U <db_username> -W -h <host> -p <port> -f fastf1_dwh.sql
```
Controleer (bv. met `pgAdmin`) of beide databases zijn aangemaakt. 

Als alles goed is gegaan, heb je nu twee databases, met daarin de juiste (lege) tabellen.

## 5.3 Stap 2: `fastf1_staging` vullen

Open in Pentaho de _job_ `load_dwh.kjb`. 

**Let op!** Er zijn twee bestanden `load_dwh`; het ene bestand (de transformatie) heeft de extensie `.ktr`. Het andere bestand (de job) heeft de extensie `.kjb`.

Volg de instructie zoals beschreven om het projectpad en de databaseparameters in te stellen en sla de job op. 

Als je de job draait en alles juist is ingesteld, vul je nu de database `fastf1_staging` met data. Merk op dat de output van het Python-script ook in rood wordt weergegeven als er géén sprake is van een foutmelding.

Open in Pentaho de transformaties:
- `driverinfo_to_staging.ktr`
- `lapspeeds_to_staging.ktr`
- `racecalendar_to_staging.ktr`

Deze transformaties zijn erg eenvoudig. Bij `lapspeeds_to_staging` is er sprake van twee tussenstappen maar verder is het een kwestie van een bron (CSV-bestand) en een bestemming (databasetabel) met elkaar verbinden. 

## 5.4 Stap 3: Sterschema ontwerpen

We volgen de stappen van Kimball om een sterschema te ontwerpen zoals behandeld in Sprint 2, week 1. 

| Stap | Omschrijving |
| --- | --- |
| 1. Proces | Het proces spreekt redelijk voor zich. Coureurs rijden alle laps van de wedstrijd. |
| 2. Granulariteit | Er is nu (in de staging) data beschikbaar voor elke lap van elke coureur van elke wedstrijd. |
| 3. Feiten | De relevante gebeurtenis is in dit geval als een coureur een lap heeft voltooid. Voor elke lap van elke coureur is er een rij. |
| 4. Meetwaarden | De volgende meetwaarden en velden: <br>- Totaal gemiddelde snelheid van de race (`raceavgspeedkmh`)<br>- Duur van lap (`time`)<br>- Leeftijd banden in ronden (`tyrelife`)<br>- Position (`position`, zou ook een dimensie kunnen zijn)<br>- Stint (`stint`, zou ook een dimensie kunnen zijn)<br>- Gemiddelde snelheid lap (`lapavgspeedkmh`) |
| 5. Dimensies | - Naam race (`racename`)<br>- Datum race (`racedate`)<br>- Coureur (`driver`)<br>- Lapnummber (`lapnumber`)<br>- Team (`team`)<br>- Positie (`position`)<br>- Stint (`stint`) |

Nagenoeg alle data hiervoor is beschikbaar in de staging-database. Alleen voor de datumdimensie gebruiken we een Excel-bestand (`data\dim_date.xlsx`). Dit bestand bevat veel extra datumkenmerken die een analyse of visualisatie kunnen vergemakkelijken of verrijken. 

Het uiteindelijke sterschema dat we gaan realiseren:

![](figures/sterschema.svg)

Elke rij in de feitentabel komt overeen met één gereden lap van één coureur (van de coureurs die de race hebben uitgereden).

# 5.5 Stap 4: Datawarehouse vullen

We vullen het datawarehouse met Pentaho-transformaties die data uit `fastf1_staging` of uit Excel lezen en wegschrijven naar `fastf1_dwh`. We gebruiken daarbij een transformatie voor elke dimensietabel en een transformatie voor de feitentabel. Alle transformaties samen roepen we aan in een andere transformatie (`load_dwh.ktr`). We gebruiken hier een transformatie voor (in plaats van een job) omdat de transformatie de mogelijkheid biedt om de uitvoering van de laatste transformatie uit te stellen tot de dimensietabellen zijn gevuld.

In de transformatie `load_dwh.ktr` roepen we ook nog twee kleine transformaties aan (`fact_lapspeeds_drop_fk.ktr` en `fact_lapspeeds_add_fk.ktr`) die de foreign keys constraints van de feitentabel verwijderen en na afloop weer toevoegen. Zonder foreign key constraints is het laadproces eenvoudiger. 

# 5.6 Stap 5: Jobs 

Voor het hele project zijn er twee jobs:

- `load_staging.kjb`. Hiermee worden alle transformaties aangeroepen die de data in de staging-database laden. 
- `load_dwh.kjb`. Hiermee worden alle transformaties aangeroepen die de data transformeren naar een sterschema en in het datawarehouse laden.
 
In beide jobs worden eerst variabelen ingesteld (bestandspad van de transformatiesbestanden en verbindingsinstellingen voor de database) zodat slechts eenmalig op een centrale plek hoeft te worden gedaan.

### Parameters/variabelen voor pad en connectie

De transformaties en de jobs moeten ook kunnen draaien op andere machines dan de machine waarop ze zijn ontwikkeld. Om dat te bereiken moet een aantal zaken configureerbaar worden. Het bestandspad van de transformaties en de Python-scripts moet met een parameter of variabele in te stellen zijn en hetzelfde geldt voor de verbinding met de database. 

In Pentaho kun je daar variabelen voor gebruiken. Deze worden gedefinieerd in de `Set variables`-stap in de jobs (`load_staging.kjb`). Om de jobs en transformaties op jouw machine te kunnen draaien moet je deze dus eerst goed instellen. Sla de job op als je de aanpassingen hebt gedaan.

Merk op dat je een transformatie in principe alleen vanuit een job kunt draaien (omdat anders de variabelen niet zijn gedefinieerd), maar met een paar kleine aanpassingen kun je de transformatie wél afzonderlijk draaien:

1. Open de job en dubbelklik op `Set variables`
2. Selecteer de variabelen door met shift op de rijkoppen (met de cijfers `1` t/m `5`) te klikken
3. Druk op <kbd>CTRL</kbd> + <kbd>C</kbd>
4. Open de transformatie
5. Druk op <kbd>CTRL</kbd> + <kbd>T</kbd>
6. Klik op het tabje `Parameters`
7. Plak de informatie met <kbd>CTRL</kbd> + <kbd>V</kbd>

# 6. Toelichting op de ETL-transformaties

## Algemene opmerkingen

Enkele algemene opmerkingen over deze toelichting en ETL:

1. De technieken die in de transformaties en de jobs worden toegepast zijn niet behandeld op college. Het vraagt de nodige technische kennis om ze goed te begrijpen. Tegelijk is ETL-software vrij laagdrempelig doordat het heel visueel georiënteerd is; je ziet als het ware de leidingen waardoor de data stroomt.Hieronder wordt een aantal zaken nader toegelicht, maar het voert te ver om alles helemaal te bespreken. 

2. Bij ETL is de volgorde waarin activiteiten worden uitgevoerd vaak erg belangrijk. Zo moeten de dimensietabellen eerst worden gevuld (of bijgewerkt) voordat de feitentabel wordt gevuld (of bijgewerkt). De feitentabel bevat namelijk voornamelijk verwijzingen (*foreign keys*) naar dimensietabellen. Om ergens naar te kunnen verwijzen moet dat waarnaar wordt verwezen wel bestaan. 
   
3. Bij ETL werk je zoveel mogelijk met *technische sleutels* die geen betekenis hebben en uit cijfers bestaan. Dat is efficiënt in de opslag maar vooral ook in de query performance. In bronapplicaties wordt vaak met zogenaamde *business keys* gewerkt waarbij vaak ook letters worden gebruikt. In de ETL worden deze sleutels vaak vervangen (of wel meegenomen maar niet gebruikt om te koppelen) door technische sleutels zonder betekenis. Daarvoor zijn er vaak opzoekacties (*lookups*) nodig om via de brondata de juiste data aan elkaar te koppelen. 
   
4. Omdat ETL-processen betrouwbaar moeten zijn, zijn ze heel kieskeurig in de datatypen die ze accepteren. Zo kunnen ze een betrouwbaar resultaat garanderen. Het nadeel is dat het ontwikkelen vaak veel tijd kost vanwege allerlei kleine detailproblemen die zich in de data voordoen. 

## `dim_date.ktr`

In de dataflow zie je twee stromen samenkomen. In de bovenste stroomt de data uit het Excel-bestand `data\dim_date.xlsx` en de onderste stroom de jaren die voorkomen in de `racecalendar`-tabel (uit de stagingdatabase). In `dim_date.xlsx` zitten datumgegevens van een groot aantal jaren maar deze stroom met een `INNER JOIN` te koppelen aan de jaren uit `racecalendar` wordt afgedwongen dat alleen relevante jaren in de datumdimensie worden geladen.

Bij bijna elke transformatie vind je wel een of meer `Select values`-operaties. Hiermee bepaal je niet alleen welke kolommen verder stroomafwaarts beschikbaar zijn, je kunt ook de namen van kolommen wijzigen en hun datatype. Vooral dat laatste is erg nuttig en belangrijk.

Merk op dat `dim_date.id` een technische sleutel is (`Integers`) maar in het getal is de datum te herkennen: `20231124` komt overeen met 24 november 2023. Dit is handig in het gebruik en het maakt het opzoeken van de sleutel makkelijker.

## `dim_driver.ktr`

Deze transformatie is eenvoudig. In de `driverinfo`-tabel zijn alle coureurs per raceseizoen opgeslagen. Een coureur kan dus meerder keren voorkomen. Daarom wordt in de bronquery (in de stap `fastf1_staging.driverinfo`) een `SELECT DISTINCT`-query gebruikt om de unieke coureurs te vinden. Vervolgens krijgen ze een technische sleutel in de stap `Add id`. 

## `dim_lap.ktr`, `dim_stint.ktr` en `dim_position.ktr`

Deze dimensies zijn heel kleine een eenvoudige dimensies. In de rapportage kan het handig zijn om ze te hebben. Er zit soms een dubbele kolom in (zowel een nummer als een tekst met hetzelfde nummer), maar juist dat kan handig zijn in de rapportage. De tekst wordt met een `formula`-component gevormd.

Bij `dim_position.ktr` zijn een paar extra stappen nodig omdat in de brondata het veld `position` ook leeg kan zijn. Voor die situatie wordt een *default*-waarde in de dimensie gereserveerd.

## `dim_race.ktr`

Deze dimensie bevat feitelijk de racekalender uit de brontabel `racecalendar`. Er wordt alleen een `id` toegevoegd en enkele kolommen worden hernoemd.

## `dim_team.ktr`

De teamdimensie is een klein lijstje dat ontstaat door een `SELECT DISTINCT` op het veld `team` uit de `lapspeeds`-tabel. In dit geval (net als bij sommige andere dimensies) moet er wel een sleutel worden toegevoegd omdat teams in de brondata nergens als afzonderlijke entiteit (met eigen primary key) zijn opgeslagen.

## `fact_lapspeeds.ktr`

De transformatie van de feitentabel is over het algemeen het meest gecompliceerd van allemaal. De basis is vaak één tabel uit de brondata (in dit geval `lapspeeds`). In dit geval komt elke rij uit de brontabel (getransformeerd) in de feitentabel terecht. 

In de stap `racedateid, season` wordt de sleutel van de datumdimensie berekend (in plaats van opgezocht in de datumdimensie), omdat berekenen efficiënter is dan opzoeken. De berekening is gebaseerd op het feit dat je een nul opschuift als je met 10 vermenigvuldigt, twee nullen als je met 100 vermenigvuldigt etc.

```
    10000 x jaar + 100 x maand + dag
```
In de stappen daarna worden `raceid`, `roundnumber` (= het volgnummer van de race in het seizoen) en `driverid_string` opgezocht. Deze laatste twee komen zelf niet in de feitentabel terecht maar ze zijn nodig om in de dimensie de technische sleutel `driverid` te vinden. Aan het eind wordt een `id` toegevoegd en worden enkele kolommen hernoemd.

# 7. Power BI

Op basis van een sterschema is het uiteindelijk heel eenvoudig om een Power BI-dashboard te maken waarmee de businessvraag wordt beantwoord:

![](figures/power_bi_dashboard.png)

Merk op dat het sterschema veel algemener is dan voor deze ene businessvraag. Er kunnen ook tal van andere businessvragen mee worden beantwoord. Stel je bent bijvoorbeeld benieuwd naar hoe voorspellend de snelste raceronde is voor het gemiddelde van de race. Dan kun je met hetzelfde sterschema de volgende visualisatie maken:

![](figures/fastest_lap_vs_race_avg.png)


# 8. Wat we NIET gedaan hebben

Zoals gezegd in de inleiding hebben we voor deze demonstratie keuzes gemaakt en een aantal onderwerpen niet of nauwelijks beschreven. Dat betekent niet dat ze niet relevant zijn voor jullie project. Het zijn onderwerpen die (met name bij **Deployment**) heel relevant kunnen zijn voor jullie project.

De volgende onderwerpen zijn niet of nauwelijks aan bod gekomen:

- **Business understanding**. De business understanding is tamelijk oppervlakkig gedaan. In werkelijkheid vraagt dit veel meer aandacht. Zie de betreffende collega's uit Sprint 1 daarover.
- **Datakwaliteit**. Feitelijk hebben we niet de data beschreven (hoeveel races, hoeveel coureurs, welke spreiding in snelheden etc.) en hebben we ook niet gekeken naar de datakwaliteit (volledigheid, actualiteit, validiteit etc.). In de praktijk is dit wel een heel belangrijk onderwerp. Veel projecten mislukken juist door te weinig aandacht voor datakwaliteit of een te lage datakwaliteit.
- **Change data capture**. In principe hebben we de ETL een beetje lui en niet zo efficiënt geïmplementeerd. Elke keer als de job draait, wordt het hele datawarehouse geleegd (alle data wordt verwijderd) en wordt de data opnieuw geladen. Dat is voor het ontwikkelen wel makkelijk maar in de praktijk is het vaak niet zo handig en efficiënt omdat er bruikbare data wordt weggegooid die vervolgens opnieuw wordt gegenereerd. In de praktijk wordt er vaak *change data capture* (CDC) toegepast, waarbij het ETL-proces alleen de wijzigingen in de bron meeneemt. De keerzijde van CDC is wel dat het veel moeilijker te implementeren is en daardoor ook foutgevoeliger.
- **Slowly changing dimensions**. In sommige situaties is het vereist dat er een versiegeschiedenis van records in dimensietabellen wordt bijgehouden. Vaak is dat juist relevant als de data niet heel sterk varieert. Je kunt bijvoorbeeld denken aan een lijst met producten van een bedrijf waarbij de attributen van de producten niet zo vaak veranderen. Om de versiegeschiedenis van de producten bij te houden, kun je werken met *slowly changing dimensions* waarbij je de verschillende versies van een product allemaal opneemt in de tabel. Je kunt daarmee nauwkeurigere analyses op de data uitvoeren maar ook dit maakt het technisch gecompliceerder.
- **Versiebeheer**. ETL-processen moeten regelmatig worden aangepast, bijvoorbeeld omdat er iets in een bronapplicatie wijzigt of omdat er andere informatie vereist is. Om dit in teamverband te doen en ervoor te zorgen dat wijzigingen traceerbaar zijn (en eventueel kunnen worden teruggedraaid) gebruiken ETL-teams meestal een vorm van versiebeheer (bv. git). 
- **Metadata**. In de Power BI-rapporten in dit project is niet te zien wanneer data is geladen/bijgewerkt. Dat kan feitelijk ook niet eenvoudig worden weergegeven omdat er tijdens het ETL-proces geen metadata wordt opgeslagen. In de praktijk moet dat zeker wel. Er moet ergens in de architectuur ruimte zijn voor logbestanden (of logdata in een tabel) zodat een gebruiker kan nagaan hoe betrouwbaar en actueel de getoonde informatie is.
- **Notificaties**. ETL-jobs en transformaties worden in de praktijk altijd geautomatiseerd uitgevoerd. Met behulp van een *job schedular* (bv. [`cron`](https://nl.wikipedia.org/wiki/Cronjob) onder Linux of de *Task scheduler* onder Windows). Onderdeel van het plannen van een taak is ook de configuratie van een notificatiesysteem. Wie of wat krijgt in welke vorm een melding als de job succesvol is uitgevoerd of als er een fout is opgetreden?
- **Python-code testen**. Voor het testen van programmacode wordt meestal gebruikgemaakt van *unit tests*, kleine specifieke stukken programmacode die je met een *test framework* (bv. `pytest`) steeds kunt draaien om de correcte werking van je code te controleren. 
- **Testen**. Een analytics-oplossing kan functioneel correct werken (knopjes, filters etc.), maar toch verkeerde informatie tonen. Hoe weten we of de geladen data juist is? Hoe stel je dat vast? Het testen van een analytics-oplossing is een vak apart. Een eerste stap is om te werken met controlegetallen (bv. het aantal rijen moet op verschillende momenten in het proces gelijk blijven of een simpele controleberekening die altijd een bepaald resultaat moet opleveren). 
- **Cloud of server**. We zijn voor dit project uitgegaan van een architectuur met een lokale pc. In de praktijk wordt vrijwel altijd gebruikgemaakt van servers binnen de infrastructuur van de organisatie of in de cloud. Dat betekent extra complexiteit door verbindingen en beveiliging. 
- **Installatie**. Als je software deployt in een nieuwe omgeving moet je de infrastructuur voorbereiden. In dit geval betekent dat installatie van (de juiste versies van) Java, Python en Pentaho.
- **Foutafhandeling**. Tijdens het ETL-proces kunnen er fouten optreden. Een bronapplicatie kan bijvoorbeeld niet bereikbaar zijn of een fout in de data kan problemen in de 'pijplijn' veroorzaken. Voor dit soort foutsituaties moeten voorzieningen worden getroffen. Fouten moeten worden gelogd, er moet een systeem of persoon worden geïnformeerd en er moet een noodvoorziening (bv. restoren van een databasebackup of rollback van de database naar een vorige toestand) in werking treden.
- **Andere bronnen**. We hebben data uit een beperkt aantal bronnen gebruikt (FastF1, Ergas en Excel). In de praktijk is het aantal bronnen vaak veel groter.
- **OTAP**. Omgeving waarin ontwikkeling, testen, acceptatie (door opdrachtgever) en productie (waarin de oplossing daadwerkelijk wordt gebruikt).

# 9. Zelf verder gaan

In dit notebook heb je een overzicht gekregen van de Python-code die in deze demonstratiecasus wordt gebruikt voor het ophalen van data uit de Fast F1 API. Daarna hebben we in detail gekeken naar de ETL-processen die je kunt toepassen om de data in een datawarehouse te laden in een sterschema. 

Hoe kun je deze kennis toepassen in je eigen project? Hieronder een paar algemene tips.

Maak vooraf voor jezelf een schets van de architectuur die je wilt gebruiken. Welke componenten, systemen en interfaces zie je? Welke data stroomt waar?

## 9.1 Tips voor Python

- **Notebooks vs. scripts**. Gebruik Jupyter Notebooks om te experimenteren en breng werkende stukjes code dan samen in scripts voor productie. 
- **Naamgeving variabelen en functies**. Gebruik duidelijke namen voor variabelen en functies. Een functie doet iets, dus een functienaam bevat meestal een werkwoord. Zorg dat de naam de lading dekt. Doe dus niet meer of minder in een functie dan de naam belooft. Een variabelenaam is vaak een ding. Zorg dat enkelvoud en meervoud van de naam overeenkomen met de inhoud van de variabele. Dus `lap` voor één ronde en `laps` voor een `list` van meerdere rondes.
- **Werk in het Engels**. Werk in je code altijd in het Engels. Gebruik bij user interfaces (in-/uitvoer) de taal van de opdrachtgever, maar hou het aan de achterkant allemaal in het Engels. 
- **Gebruik commentaar**. Gebruik commentaar (vooral in docstrings van functies en klassen: `"""..."""`) Probeer je code zo duidelijk te schrijven dat de code zichzelf uitlegt. Verantwoord eventuele keuzes in commentaar. Gebruik commentaar dus voor het waarom en niet voor het hoe.
- **Gebruik functies**. Gebruik functies om grotere problemen in kleinere deelproblemen te splitsen. Functies zijn eenvoudiger te testen.
- **Gebruik kleine functies**. Splits grotere functie op in kleinere met specifieke taken. Dat maakt ze overzichtelijker en maakt debuggen en aanpassen van code eenvoudiger.
- **Gebruik de debugger**. Gebruik de debugfunctionaliteit in VSCode of PyCharm. Dat helpt ontzettend veel bij het begrijpen van het gedrag van je code en het vinden van bugs. Ga na wat een *breakpoint* is, hoe je een *watch* toevoegt en hoe je door code stapt. Ga na wat het verschil tussen een functie uitvoeren de functie induiken. Leer daarbij ook de sneltoetsen (bv. `F5`, `Shift`+`F5`, `F10` en `F11` in VSCode).
- **Google en Stackoverflow**. *Google is your friend*. Zoek specifiek en probeer daarbij de juiste terminologie te hanteren. Wees altijd kritisch over wat je leest (lang niet alles klopt of is nog actueel). Neem stukjes code alleen over als je de voorgestelde oplossing zelf begrijpt (zie volgende tip).
- **ChatGPT**. De kennis van ChatGPT van Python is wisselend. Niet alles wat de tool voorstelt, klopt of is bruikbaar. Echter, jee kunt de tool wel uitstekend gebruiken om code uit te leggen (zie vorige tip).
- **git**. Gebruik git om samen te werken en je code te backuppen. 
- **Deployment**. Hou rekening met deployment. Zorg dat je zaken configureerbaar zijn en hou de configuratie centraal (zie `settings.py`).

## 7.2 Tips voor het gebruik van ETL-tools

Pentaho Data Integration (community edition) is een uitstekende professionele tool. Echter, het is een tool die is gericht op gebruik in een automatiseringscontext. Hier en daar voelt het soms wat logger en 'industriëler' aan dan een consumentenapplicatie. 

Er zijn ook diverse andere ETL-tools: SQL Server Integration Services (SSIS) en Azure Data Factory (beide van Microsoft). Veel tools hebben soortgelijke concepten en werkwijzen. Veel tools bieden ook ETL-functionaliteit maar zijn er niet primair voor bedoeld. Probeer tools vooral te gebruiken voor hetgeen waarvoor ze zijn bedoeld. Power BI is bedoeld voor rapportage en niet voor ETL, hoewel je er ook ETL-functionaliteit in vindt.

